<a href="https://colab.research.google.com/github/basharE/CVDs_Predictors/blob/main/CVDs_Predictors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing packages/Imports

In [1]:
!pip install -U -q PyDrive
!pip install shap
!pip install boruta
!pip install BorutaShap
!pip install catboost
# !pip install verstack

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import IsolationForest
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import r2_score as R2
from scipy.special import softmax
from boruta import BorutaPy
from BorutaShap import BorutaShap
# from verstack import LGBMTuner
from sklearn.model_selection import cross_val_score

import pandas as pd
import numpy as np
import seaborn as sn
sn.set_style(style='darkgrid')
import matplotlib.pyplot as plt
import math

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 569 kB 34.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 56 kB 4.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.2 MB/s 


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Aug 20 09:31:48 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


# Uploading data 

## Connecting to google drive

In [4]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

## Uploading csv files

In [5]:
df_Cleveland = pd.read_csv("/content/drive/MyDrive/CVD_DATA/cleveland.data", delimiter=" ")
df_Hungarian = pd.read_csv("/content/drive/MyDrive/CVD_DATA/hungarian.data", delimiter=" ")
df_Switzerland = pd.read_csv("/content/drive/MyDrive/CVD_DATA/switzerland.data", delimiter=" ")
df_LongBeachVA = pd.read_csv("/content/drive/MyDrive/CVD_DATA/long-beach-va.data", delimiter=" ")

In [6]:
df = pd.concat([df_Cleveland, df_Hungarian, df_Switzerland, df_LongBeachVA])

In [7]:
# df_co = df[['smoke'
# ,'cigs'
# ,'years'
# ,'age'      
# ,'sex'       
# ,'cp'        
# ,'trestbps'
# ,'chol'   
# ,'fbs'       
# ,'restecg'
# ,'thalach'   
# ,'exang'    
# ,'oldpeak'
# ,'slope'    
# ,'ca'      
# ,'thal'
# ,'num']]

In [8]:
df_co = df

In [9]:
conditions = [
    (df_co['smoke'] == -9) & (df_co['cigs'] == -9) & (df_co['years'] == -9),
    (df_co['smoke'] == 0) | (df_co['cigs'] == 0) | (df_co['years'] == 0),
    (df_co['smoke'] > 0) | (df_co['cigs'] > 0) | (df_co['years'] > 0)
]

values = [-9, 0, 1]

df_co['is_smoker'] = np.select(conditions, values)

In [10]:
conditions = [
    (df_co['num'] > 0),
    (df_co['num'] == 0)
]

values = [1, 0]

df_co['num_new'] = np.select(conditions, values)

## Correlation

In [11]:
df_smoker = df_co[['is_smoker', 'num', 'num_new']]

In [12]:
df_smoker = df_smoker[df_smoker['is_smoker'] != -9]

In [13]:
df_smoker

,is_smoker,num,num_new
0,1,0,0
1,1,2,1
2,1,1,1
3,0,0,0
4,0,0,0
...,...,...,...
195,0,1,1
196,1,0,0
197,1,2,1
198,0,0,0


In [14]:
#########################################################
# Cross tabulation between is_smoker and num
CrosstabResult=pd.crosstab(index=df_smoker['num'],columns=df_smoker['is_smoker'])
print(CrosstabResult)
 
# importing the required function
from scipy.stats import chi2_contingency
 
# Performing Chi-sq test
ChiSqResult = chi2_contingency(CrosstabResult)
 
# P-Value is the Probability of H0 being True
# If P-Value&gt;0.05 then only we Accept the assumption(H0)
 
print('The P-Value of the ChiSq Test is:', ChiSqResult[1])

stat, p, dof, expected = chi2_contingency(CrosstabResult)

print('The P-Value of the ChiSq Test is:', ChiSqResult)
print('stat:', stat)
print('p:', p)
print('dof:', dof)
print('expected:', expected)

is_smoker    0    1
num                
0          106  109
1           47   70
2           32   46
3           37   42
4            4   19
The P-Value of the ChiSq Test is: 0.03588940652959348
The P-Value of the ChiSq Test is: (10.28510734629305, 0.03588940652959348, 4, array([[ 94.90234375, 120.09765625],
       [ 51.64453125,  65.35546875],
       [ 34.4296875 ,  43.5703125 ],
       [ 34.87109375,  44.12890625],
       [ 10.15234375,  12.84765625]]))
stat: 10.28510734629305
p: 0.03588940652959348
dof: 4
expected: [[ 94.90234375 120.09765625]
 [ 51.64453125  65.35546875]
 [ 34.4296875   43.5703125 ]
 [ 34.87109375  44.12890625]
 [ 10.15234375  12.84765625]]


In [15]:
from scipy.stats import chi2

# interpret test-statistic
prob = 0.95
critical = chi2.ppf(prob, dof)
if abs(stat) >= critical:
	print('Dependent (reject H0)')
else:
	print('Independent (fail to reject H0)')

Dependent (reject H0)


In [16]:

# interpret p-value
alpha = 1.0 - prob
if p <= alpha:
	print('Dependent (reject H0)')
else:
	print('Independent (fail to reject H0)')

Dependent (reject H0)


In [17]:
#########################################################
# Cross tabulation between is_smoker and num
CrosstabResult=pd.crosstab(index=df_smoker['is_smoker'],columns=df_smoker['num_new'])
print(CrosstabResult)
 
# importing the required function
from scipy.stats import chi2_contingency
 
# Performing Chi-sq test
ChiSqResult = chi2_contingency(CrosstabResult)
 
# P-Value is the Probability of H0 being True
# If P-Value&gt;0.05 then only we Accept the assumption(H0)
 
print('The P-Value of the ChiSq Test is:', ChiSqResult)

num_new      0    1
is_smoker          
0          106  120
1          109  177
The P-Value of the ChiSq Test is: (3.6522471454911725, 0.05599361583364363, 1, array([[ 94.90234375, 131.09765625],
       [120.09765625, 165.90234375]]))


## Feature Importance


In [18]:
df_co

,id,ccf,age,sex,painloc,painexer,relrest,pncaden,cp,trestbps,...,lvx1,lvx2,vx3,lvx4,lvf,cathef,junk,name,is_smoker,num_new
0,1,0,63,1,-9,-9,-9,-9,1,145,...,1,1,1,1,1,-9.00,-9.0,name,1,0
1,2,0,67,1,-9,-9,-9,-9,4,160,...,1,1,1,1,1,-9.00,-9.0,name,1,1
2,3,0,67,1,-9,-9,-9,-9,4,120,...,1,1,1,7,3,-9.00,-9.0,name,1,1
3,4,0,37,1,-9,-9,-9,-9,3,130,...,1,1,1,1,1,-9.00,-9.0,name,0,0
4,6,0,41,0,-9,-9,-9,-9,2,130,...,1,1,1,1,1,-9.00,-9.0,name,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,200,0,54,0,1,1,1,-9,4,127,...,1,1,1,1,1,0.76,5.6,name,0,1
196,201,0,62,1,0,0,0,-9,1,-9,...,1,1,1,1,2,0.62,3.5,name,1,0
197,202,0,55,1,1,1,1,-9,4,122,...,1,1,1,1,1,0.69,5.6,name,1,1
198,116,0,58,1,1,1,1,-9,4,-9,...,1,1,1,1,1,0.81,6.0,name,0,0


In [19]:
columns_to_delete = ['smoke', 'cigs', 'years', 'num_new', 'name']
df_co = df_co.drop(columns_to_delete, axis=1)

In [20]:
# df_co = df_co[df_co['age'] != -9]
# df_co = df_co[df_co['sex'] != -9]
# df_co = df_co[df_co['cp'] != -9]
# df_co = df_co[df_co['trestbps'] != -9]
# df_co = df_co[df_co['chol'] != -9]
# df_co = df_co[df_co['fbs'] != -9]
# df_co = df_co[df_co['restecg'] != -9]
# df_co = df_co[df_co['thalach'] != -9]
# df_co = df_co[df_co['exang'] != -9]
# df_co = df_co[df_co['oldpeak'] != -9]
# df_co = df_co[df_co['slope'] != -9]
# df_co = df_co[df_co['ca'] != -9]
# df_co = df_co[df_co['thal'] != -9]
# df_co = df_co[df_co['num'] != -9]
# df_co = df_co[df_co['is_smoker'] != -9]

In [21]:
df_co

,id,ccf,age,sex,painloc,painexer,relrest,pncaden,cp,trestbps,...,rcaprox,rcadist,lvx1,lvx2,vx3,lvx4,lvf,cathef,junk,is_smoker
0,1,0,63,1,-9,-9,-9,-9,1,145,...,1,1,1,1,1,1,1,-9.00,-9.0,1
1,2,0,67,1,-9,-9,-9,-9,4,160,...,1,1,1,1,1,1,1,-9.00,-9.0,1
2,3,0,67,1,-9,-9,-9,-9,4,120,...,2,2,1,1,1,7,3,-9.00,-9.0,1
3,4,0,37,1,-9,-9,-9,-9,3,130,...,1,1,1,1,1,1,1,-9.00,-9.0,0
4,6,0,41,0,-9,-9,-9,-9,2,130,...,1,1,1,1,1,1,1,-9.00,-9.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,200,0,54,0,1,1,1,-9,4,127,...,1,1,1,1,1,1,1,0.76,5.6,0
196,201,0,62,1,0,0,0,-9,1,-9,...,1,1,1,1,1,1,2,0.62,3.5,1
197,202,0,55,1,1,1,1,-9,4,122,...,2,1,1,1,1,1,1,0.69,5.6,1
198,116,0,58,1,1,1,1,-9,4,-9,...,1,1,1,1,1,1,1,0.81,6.0,0


In [22]:
def move_column_inplace(df, col, pos):
    col = df.pop(col)
    df.insert(pos, col.name, col)

In [23]:
move_column_inplace(df_co,"num",0)

In [24]:
df_co

,num,id,ccf,age,sex,painloc,painexer,relrest,pncaden,cp,...,rcaprox,rcadist,lvx1,lvx2,vx3,lvx4,lvf,cathef,junk,is_smoker
0,0,1,0,63,1,-9,-9,-9,-9,1,...,1,1,1,1,1,1,1,-9.00,-9.0,1
1,2,2,0,67,1,-9,-9,-9,-9,4,...,1,1,1,1,1,1,1,-9.00,-9.0,1
2,1,3,0,67,1,-9,-9,-9,-9,4,...,2,2,1,1,1,7,3,-9.00,-9.0,1
3,0,4,0,37,1,-9,-9,-9,-9,3,...,1,1,1,1,1,1,1,-9.00,-9.0,0
4,0,6,0,41,0,-9,-9,-9,-9,2,...,1,1,1,1,1,1,1,-9.00,-9.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,1,200,0,54,0,1,1,1,-9,4,...,1,1,1,1,1,1,1,0.76,5.6,0
196,0,201,0,62,1,0,0,0,-9,1,...,1,1,1,1,1,1,2,0.62,3.5,1
197,2,202,0,55,1,1,1,1,-9,4,...,2,1,1,1,1,1,1,0.69,5.6,1
198,0,116,0,58,1,1,1,1,-9,4,...,1,1,1,1,1,1,1,0.81,6.0,0


In [25]:
def corr_matrix_vis(df):
    fig, ax = plt.subplots(figsize=(150, 150))

    corrMatrix = df.corr()
    sn.heatmap(corrMatrix, annot=True, cmap="PiYG")
    plt.show()

In [26]:
corr_matrix_vis(df_co)

In [27]:
s = df_co.corr()

In [28]:
s['num'].sort_values(ascending=False)

num        1.000000
cp         0.397565
cxmain     0.376215
rcaprox    0.367561
ladprox    0.346967
             ...   
thaldur   -0.238011
thalach   -0.256464
ccf             NaN
pncaden         NaN
restckm         NaN
Name: num, Length: 73, dtype: float64

In [29]:
def print_feature_importances_random_forest(random_forest_model):
    
    '''
    Prints the feature importances of a Random Forest model in an ordered way.
    random_forest_model -> The sklearn.ensemble.RandomForestRegressor or RandomForestClassifier trained model
    '''
    
    # Fetch the feature importances and feature names
    importances = random_forest_model.feature_importances_
    features = random_forest_model.feature_names_in_
    
    # Organize them in a dictionary
    feature_importances = {fea: imp for imp, fea in zip(importances, features)}
    
    # Sorts the dictionary
    feature_importances = {k: v for k, v in sorted(feature_importances.items(), key=lambda item: item[1], reverse = True)}
    
    # Prints the feature importances
    for k, v in feature_importances.items():
        print(f"{k} -> {v:.4f}")

In [30]:
def print_feature_importances_shap_values(shap_values, features):
    
    '''
    Prints the feature importances based on SHAP values in an ordered way
    shap_values -> The SHAP values calculated from a shap.Explainer object
    features -> The name of the features, on the order presented to the explainer
    '''

    # Calculates the feature importance (mean absolute shap value) for each feature
    importances = []
    for i in range(shap_values.values.shape[1]):
        importances.append(np.mean(np.abs(shap_values.values[:, i])))
        
    # Calculates the normalized version
    importances_norm = softmax(importances)

    # Organize the importances and columns in a dictionary
    feature_importances = {fea: imp for imp, fea in zip(importances, features)}
    feature_importances_norm = {fea: imp for imp, fea in zip(importances_norm, features)}

    # Sorts the dictionary
    feature_importances = {k: v for k, v in sorted(feature_importances.items(), key=lambda item: item[1], reverse = True)}
    feature_importances_norm= {k: v for k, v in sorted(feature_importances_norm.items(), key=lambda item: item[1], reverse = True)}

    # Prints the feature importances
    for k, v in feature_importances.items():
        print(f"{k} -> {v:.4f} (softmax = {feature_importances_norm[k]:.4f})")


In [31]:
#first we split our data into input and output
# y is the output and is stored in "Class" column of dataframe
# X contains the other columns and are features or input
y = df_co.num
df_co.drop(['num'], axis=1, inplace=True)
X = df_co
 
# Now we split the dataset in train and test part
# here the train set is 80% and test set is 20%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [32]:
# Prepares a default instance of the random forest regressor
model = RandomForestRegressor()
# Fits the model on the training data
model.fit(X_train, y_train)
RandomForestRegressor()
# Evaluates the model
y_pred = model.predict(X_test)

In [33]:
# Prints the feature importances
print_feature_importances_random_forest(model)

rcaprox -> 0.3679
rcadist -> 0.1654
lmt -> 0.1484
ladprox -> 0.0909
cxmain -> 0.0761
laddist -> 0.0729
om1 -> 0.0253
ramus -> 0.0088
oldpeak -> 0.0043
restef -> 0.0026
fbs -> 0.0024
age -> 0.0024
id -> 0.0020
thalach -> 0.0018
thaltime -> 0.0017
chol -> 0.0015
dm -> 0.0014
met -> 0.0014
rldv5 -> 0.0013
ekgyr -> 0.0013
thalrest -> 0.0013
rldv5e -> 0.0010
om2 -> 0.0010
cday -> 0.0009
tpeakbps -> 0.0009
cmo -> 0.0009
diag -> 0.0008
proto -> 0.0008
lvf -> 0.0008
trestbpd -> 0.0007
ekgday -> 0.0007
slope -> 0.0007
cp -> 0.0006
thalsev -> 0.0006
thaldur -> 0.0006
tpeakbpd -> 0.0006
restwm -> 0.0006
ekgmo -> 0.0006
cathef -> 0.0006
cyr -> 0.0005
prop -> 0.0005
dummy -> 0.0004
lvx4 -> 0.0004
exang -> 0.0003
vx3 -> 0.0003
thal -> 0.0003
trestbps -> 0.0003
diuretic -> 0.0003
lvx2 -> 0.0003
ca -> 0.0003
htn -> 0.0003
famhist -> 0.0002
restecg -> 0.0002
painexer -> 0.0001
is_smoker -> 0.0001
exerwm -> 0.0001
xhypo -> 0.0001
nitr -> 0.0001
junk -> 0.0001
relrest -> 0.0001
thalpul -> 0.0001
dig -> 0

## Feature Importance/Selection for Overfitting Handling

In [34]:
#first we split our data into input and output
# y is the output and is stored in "Class" column of dataframe
# X contains the other columns and are features or input
lines_number = 1000
y_feature_importance = y.head(lines_number)
X_feature_importance = X.head(lines_number)
 
# Now we split the dataset in train and test part
# here the train set is 75% and test set is 25%
X_train_fi, X_test_fi, y_train_fi, y_test_fi = train_test_split(X_feature_importance, y_feature_importance, test_size=0.25, random_state=2)

In [35]:
import shap
# Fits the explainer
explainer = shap.Explainer(model.predict, X_feature_importance)
# Calculates the SHAP values - It takes some time
shap_values = explainer(X_test_fi)
# Prints the SHAP feature importances
print_feature_importances_shap_values(shap_values, X_feature_importance.columns)

Permutation explainer: 226it [00:42,  4.42it/s]

rcaprox -> 0.3875 (softmax = 0.0200)
ladprox -> 0.2812 (softmax = 0.0179)
cxmain -> 0.2498 (softmax = 0.0174)
laddist -> 0.2188 (softmax = 0.0169)
rcadist -> 0.1504 (softmax = 0.0157)
lmt -> 0.1178 (softmax = 0.0152)
om1 -> 0.1082 (softmax = 0.0151)
ramus -> 0.0172 (softmax = 0.0138)
oldpeak -> 0.0067 (softmax = 0.0136)
fbs -> 0.0050 (softmax = 0.0136)
thalach -> 0.0034 (softmax = 0.0136)
om2 -> 0.0030 (softmax = 0.0136)
rldv5e -> 0.0029 (softmax = 0.0136)
rldv5 -> 0.0028 (softmax = 0.0136)
id -> 0.0024 (softmax = 0.0136)
tpeakbps -> 0.0023 (softmax = 0.0136)
ekgyr -> 0.0023 (softmax = 0.0136)
thalrest -> 0.0022 (softmax = 0.0136)
age -> 0.0020 (softmax = 0.0136)
chol -> 0.0018 (softmax = 0.0136)
met -> 0.0016 (softmax = 0.0136)
thaltime -> 0.0015 (softmax = 0.0136)
proto -> 0.0015 (softmax = 0.0136)
cathef -> 0.0014 (softmax = 0.0136)
diuretic -> 0.0014 (softmax = 0.0136)
tpeakbpd -> 0.0013 (softmax = 0.0136)
cmo -> 0.0012 (softmax = 0.0136)
cp -> 0.0012 (softmax = 0.0136)
thalsev -> 

### Select features using Boruta


In [36]:
# Default title text
# Defines the estimator used by the Boruta algorithm
estimator = RandomForestRegressor()

In [37]:
# Creates the BorutaPy object
boruta = BorutaPy(estimator = estimator, n_estimators = 'auto', max_iter = 100)

In [38]:
# Fits Boruta
boruta.fit(np.array(X_train_fi), np.array(y_train_fi))

BorutaPy(estimator=RandomForestRegressor(n_estimators=50,
                                         random_state=RandomState(MT19937) at 0x7FBDD8E8D490),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x7FBDD8E8D490)

In [39]:
# Important features
important = list(X_feature_importance.columns[boruta.support_])
print(f"Features confirmed as important: {important}")

# Tentative features
tentative = list(X_feature_importance.columns[boruta.support_weak_])
print(f"Unconfirmed features (tentative): {tentative}")

# Unimportant features
unimportant = list(X_feature_importance.columns[~(boruta.support_ | boruta.support_weak_)])
print(f"Features confirmed as unimportant: {unimportant}")

Features confirmed as important: ['rldv5', 'lmt', 'ladprox', 'laddist', 'cxmain', 'ramus', 'om1', 'rcaprox', 'rcadist']
Unconfirmed features (tentative): ['age', 'thalach', 'tpeakbps']
Features confirmed as unimportant: ['id', 'ccf', 'sex', 'painloc', 'painexer', 'relrest', 'pncaden', 'cp', 'trestbps', 'htn', 'chol', 'fbs', 'dm', 'famhist', 'restecg', 'ekgmo', 'ekgday', 'ekgyr', 'dig', 'prop', 'nitr', 'pro', 'diuretic', 'proto', 'thaldur', 'thaltime', 'met', 'thalrest', 'tpeakbpd', 'dummy', 'trestbpd', 'exang', 'xhypo', 'oldpeak', 'slope', 'rldv5e', 'ca', 'restckm', 'exerckm', 'restef', 'restwm', 'exeref', 'exerwm', 'thal', 'thalsev', 'thalpul', 'earlobe', 'cmo', 'cday', 'cyr', 'diag', 'om2', 'lvx1', 'lvx2', 'vx3', 'lvx4', 'lvf', 'cathef', 'junk', 'is_smoker']


In [40]:
# Removes them
X_train_boruta = X_train.drop(columns = unimportant)
X_test_boruta = X_test.drop(columns = unimportant)

In [41]:
X_train_boruta.head()

,age,thalach,tpeakbps,rldv5,lmt,ladprox,laddist,cxmain,ramus,om1,rcaprox,rcadist
99,63,160,180,31,1,1,1,1,1,1,1,1
254,62,152,240,12,-9,-9,-9,-9,-9,-9,-9,-9
54,53,106,145,20,1,1,2,1,1,1,1,1
108,50,135,184,12,1,-9,-9,-9,1,-9,-9,-9
118,63,132,150,-9,1,1,2,2,-9,2,2,1


### Select features using Boruta-SHAP


In [42]:
# Creates a BorutaShap selector for regression
selector = BorutaShap(importance_measure = 'shap', classification = True)

In [43]:
# Fits the selector
selector.fit(X = X_train_fi, y = y_train_fi, n_trials = 100, sample = True, verbose = True)
# n_trials -> number of iterations for Boruta algorithm
# sample -> samples the data so it goes faster

  0%|          | 0/100 [00:00<?, ?it/s]

22 attributes confirmed important: ['rcaprox', 'exang', 'chol', 'laddist', 'cp', 'oldpeak', 'rcadist', 'diag', 'thalach', 'ramus', 'thal', 'slope', 'lmt', 'om1', 'lvx4', 'id', 'ladprox', 'relrest', 'om2', 'thaltime', 'cxmain', 'painexer']
48 attributes confirmed unimportant: ['thalpul', 'sex', 'ca', 'restef', 'ekgyr', 'age', 'met', 'proto', 'rldv5', 'cathef', 'htn', 'painloc', 'trestbps', 'cday', 'thalsev', 'tpeakbpd', 'cmo', 'is_smoker', 'pro', 'earlobe', 'nitr', 'ekgmo', 'famhist', 'restckm', 'xhypo', 'ekgday', 'dm', 'restecg', 'pncaden', 'thalrest', 'exerckm', 'dummy', 'restwm', 'trestbpd', 'lvx1', 'diuretic', 'dig', 'vx3', 'lvf', 'lvx2', 'rldv5e', 'prop', 'fbs', 'exerwm', 'exeref', 'ccf', 'cyr', 'junk']
2 tentative attributes remains: ['tpeakbps', 'thaldur']


In [44]:
# Display features to be removed
features_to_remove = selector.features_to_remove
print(features_to_remove)

['ccf' 'age' 'sex' 'painloc' 'pncaden' 'trestbps' 'htn' 'fbs' 'dm'
 'famhist' 'restecg' 'ekgmo' 'ekgday' 'ekgyr' 'dig' 'prop' 'nitr' 'pro'
 'diuretic' 'proto' 'met' 'thalrest' 'tpeakbpd' 'dummy' 'trestbpd' 'xhypo'
 'rldv5' 'rldv5e' 'ca' 'restckm' 'exerckm' 'restef' 'restwm' 'exeref'
 'exerwm' 'thalsev' 'thalpul' 'earlobe' 'cmo' 'cday' 'cyr' 'lvx1' 'lvx2'
 'vx3' 'lvf' 'cathef' 'junk' 'is_smoker']


In [123]:
features_to_remove_new = np.delete(features_to_remove, 47)

print(features_to_remove)

['ccf' 'age' 'sex' 'painloc' 'pncaden' 'trestbps' 'htn' 'fbs' 'dm'
 'famhist' 'restecg' 'ekgmo' 'ekgday' 'ekgyr' 'dig' 'prop' 'nitr' 'pro'
 'diuretic' 'proto' 'met' 'thalrest' 'tpeakbpd' 'dummy' 'trestbpd' 'xhypo'
 'rldv5' 'rldv5e' 'ca' 'restckm' 'exerckm' 'restef' 'restwm' 'exeref'
 'exerwm' 'thalsev' 'thalpul' 'earlobe' 'cmo' 'cday' 'cyr' 'lvx1' 'lvx2'
 'vx3' 'lvf' 'cathef' 'junk' 'is_smoker']


In [124]:
features_to_remove_new

array(['ccf', 'age', 'sex', 'painloc', 'pncaden', 'trestbps', 'htn',
       'fbs', 'dm', 'famhist', 'restecg', 'ekgmo', 'ekgday', 'ekgyr',
       'dig', 'prop', 'nitr', 'pro', 'diuretic', 'proto', 'met',
       'thalrest', 'tpeakbpd', 'dummy', 'trestbpd', 'xhypo', 'rldv5',
       'rldv5e', 'ca', 'restckm', 'exerckm', 'restef', 'restwm', 'exeref',
       'exerwm', 'thalsev', 'thalpul', 'earlobe', 'cmo', 'cday', 'cyr',
       'lvx1', 'lvx2', 'vx3', 'lvf', 'cathef', 'junk'], dtype=object)

In [125]:
# Removes them
X_train_boruta_shap = X_train.drop(columns = features_to_remove)
X_test_boruta_shap = X_test.drop(columns = features_to_remove)

In [126]:
# Removes them
X_train_boruta_shap_with_smoking = X_train.drop(columns = features_to_remove_new)
X_test_boruta_shap_with_smoking = X_test.drop(columns = features_to_remove_new)

In [130]:
X_train_boruta_shap

,id,painexer,relrest,cp,chol,thaldur,thaltime,thalach,tpeakbps,exang,...,ladprox,laddist,diag,cxmain,ramus,om1,om2,rcaprox,rcadist,lvx4
99,100,0,-9,3,0,6.0,-9.0,160,180,0,...,1,1,1,1,1,1,1,1,1,1
254,1017,0,0,2,271,12.0,11.0,152,240,0,...,-9,-9,-9,-9,-9,-9,-9,-9,-9,1
54,55,1,1,4,0,4.0,-9.0,106,145,0,...,1,2,1,1,1,1,1,1,1,1
108,1234,1,1,4,129,14.0,-9.0,135,184,0,...,-9,-9,-9,-9,1,-9,-9,-9,-9,1
118,128,-9,-9,4,330,7.5,6.0,132,150,1,...,1,2,-9,2,-9,2,-9,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,113,-9,-9,4,177,7.0,3.0,162,165,1,...,2,1,-9,2,-9,1,-9,1,1,1
270,287,-9,-9,4,207,9.0,-9.0,138,170,1,...,1,2,-9,1,-9,1,-9,1,1,1
161,166,1,1,4,369,-9.0,-9.0,-9,-9,-9,...,2,1,1,1,1,2,1,2,1,1
153,1147,1,1,4,270,12.0,-9.0,140,190,0,...,-9,-9,-9,-9,-9,-9,-9,-9,-9,1


In [129]:
X_train_boruta_shap_with_smoking

,id,painexer,relrest,cp,chol,thaldur,thaltime,thalach,tpeakbps,exang,...,laddist,diag,cxmain,ramus,om1,om2,rcaprox,rcadist,lvx4,is_smoker
99,100,0,-9,3,0,6.0,-9.0,160,180,0,...,1,1,1,1,1,1,1,1,1,1
254,1017,0,0,2,271,12.0,11.0,152,240,0,...,-9,-9,-9,-9,-9,-9,-9,-9,1,-9
54,55,1,1,4,0,4.0,-9.0,106,145,0,...,2,1,1,1,1,1,1,1,1,0
108,1234,1,1,4,129,14.0,-9.0,135,184,0,...,-9,-9,-9,1,-9,-9,-9,-9,1,-9
118,128,-9,-9,4,330,7.5,6.0,132,150,1,...,2,-9,2,-9,2,-9,2,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,113,-9,-9,4,177,7.0,3.0,162,165,1,...,1,-9,2,-9,1,-9,1,1,1,1
270,287,-9,-9,4,207,9.0,-9.0,138,170,1,...,2,-9,1,-9,1,-9,1,1,1,1
161,166,1,1,4,369,-9.0,-9.0,-9,-9,-9,...,1,1,1,1,2,1,2,1,1,1
153,1147,1,1,4,270,12.0,-9.0,140,190,0,...,-9,-9,-9,-9,-9,-9,-9,-9,1,-9


In [47]:
# 'smoke' --
# ,'cigs'
# ,'years'
# ,'age'      
# ,'sex'       
# ,'cp'        
# ,'trestbps'
# ,'chol'   
# ,'fbs'       
# ,'restecg'
# ,'thalach'   
# ,'exang'    
# ,'oldpeak'
# ,'slope'    
# ,'ca'      
# ,'thal'
# ,'num']]

In [113]:
!pip install tabulate


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [161]:
from tabulate import tabulate


In [162]:
all_data = [["Classifier", "Without_Smoking", "With_Smoking"],
            [],
            [],
            [],
            [],
            []]

In [163]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost.sklearn import XGBClassifier

from sklearn.metrics import f1_score

In [164]:
def DecisionTreeClassifier_model(x_train, x_test, y_train, y_test):
  # Training the model is as simple as this
  # Use the function imported above and apply fit() on it
  model = DecisionTreeClassifier()
  model .fit(x_train,y_train)
  # We use the predict() on the model to predict the output
  pred=model .predict(x_test)
  # for classification we use accuracy and F1 score
  all_data[1].insert(0,'DecisionTreeClassifier')
  x = f1_score(y_test, pred, average='weighted')
  all_data[1].insert(1,x)
  print(x)

In [165]:
def DecisionTreeClassifier_model_(x_train, x_test, y_train, y_test):
  # Training the model is as simple as this
  # Use the function imported above and apply fit() on it
  model = DecisionTreeClassifier()
  model .fit(x_train,y_train)
  # We use the predict() on the model to predict the output
  pred=model .predict(x_test)
  # for classification we use accuracy and F1 score
  x = f1_score(y_test, pred, average='weighted')
  all_data[1].insert(2,x)
  print(x)

In [166]:
def RandomForestClassifier_model(x_train, x_test, y_train, y_test):
  # Training the model is as simple as this
  # Use the function imported above and apply fit() on it
  model = RandomForestClassifier()
  model .fit(x_train,y_train)
  # We use the predict() on the model to predict the output
  pred=model .predict(x_test)
  # for classification we use accuracy and F1 score
  all_data[2].insert(0,'RandomForestClassifier')
  x = f1_score(y_test, pred, average='weighted')
  all_data[2].insert(1,x)
  print(x)

In [167]:
def RandomForestClassifier_model_(x_train, x_test, y_train, y_test):
  # Training the model is as simple as this
  # Use the function imported above and apply fit() on it
  model = RandomForestClassifier()
  model .fit(x_train,y_train)
  # We use the predict() on the model to predict the output
  pred=model .predict(x_test)
  # for classification we use accuracy and F1 score
  x = f1_score(y_test, pred, average='weighted')
  all_data[2].insert(2,x)
  print(x)

In [168]:
def GradientBoostingClassifier_model(x_train, x_test, y_train, y_test):
  # Training the model is as simple as this
  # Use the function imported above and apply fit() on it
  model = GradientBoostingClassifier()
  model .fit(x_train,y_train)
  # We use the predict() on the model to predict the output
  pred=model .predict(x_test)
  # for classification we use accuracy and F1 score
  all_data[3].insert(0,'GradientBoostingClassifier')
  x = f1_score(y_test, pred, average='weighted')
  all_data[3].insert(1,x)
  print(x)

In [169]:
def GradientBoostingClassifier_model_(x_train, x_test, y_train, y_test):
  # Training the model is as simple as this
  # Use the function imported above and apply fit() on it
  model = GradientBoostingClassifier()
  model .fit(x_train,y_train)
  # We use the predict() on the model to predict the output
  pred=model .predict(x_test)
  # for classification we use accuracy and F1 score
  x = f1_score(y_test, pred, average='weighted')
  all_data[3].insert(2,x)
  print(x)

In [170]:
def LGBMClassifier_model(x_train, x_test, y_train, y_test):
  # Training the model is as simple as this
  # Use the function imported above and apply fit() on it
  model = LGBMClassifier()
  model .fit(x_train,y_train)
  # We use the predict() on the model to predict the output
  pred=model .predict(x_test)
  # for classification we use accuracy and F1 score
  all_data[4].insert(0,'LGBMClassifier')
  x = f1_score(y_test, pred, average='weighted')
  all_data[4].insert(1,x)
  print(x)

In [171]:
def LGBMClassifier_model_(x_train, x_test, y_train, y_test):
  # Training the model is as simple as this
  # Use the function imported above and apply fit() on it
  model = LGBMClassifier()
  model .fit(x_train,y_train)
  # We use the predict() on the model to predict the output
  pred=model .predict(x_test)
  # for classification we use accuracy and F1 score
  x = f1_score(y_test, pred, average='weighted')
  all_data[4].insert(2,x)
  print(x)

In [172]:
def XGBClassifier_model(x_train, x_test, y_train, y_test):
  # Training the model is as simple as this
  # Use the function imported above and apply fit() on it
  model = XGBClassifier()
  model .fit(x_train,y_train)
  # We use the predict() on the model to predict the output
  pred=model .predict(x_test)
  # for classification we use accuracy and F1 score
  all_data[5].insert(0,'XGBClassifier_model')
  x = f1_score(y_test, pred, average='weighted')
  all_data[5].insert(1,x)
  print(x)

In [173]:
def XGBClassifier_model_(x_train, x_test, y_train, y_test):
  # Training the model is as simple as this
  # Use the function imported above and apply fit() on it
  model = XGBClassifier()
  model .fit(x_train,y_train)
  # We use the predict() on the model to predict the output
  pred=model .predict(x_test)
  # for classification we use accuracy and F1 score
  x = f1_score(y_test, pred, average='weighted')
  all_data[5].insert(2,x)
  print(x)

In [174]:
DecisionTreeClassifier_model(X_train_boruta_shap, X_test_boruta_shap, y_train, y_test)
RandomForestClassifier_model(X_train_boruta_shap, X_test_boruta_shap, y_train, y_test)
GradientBoostingClassifier_model(X_train_boruta_shap, X_test_boruta_shap, y_train, y_test)
LGBMClassifier_model(X_train_boruta_shap, X_test_boruta_shap, y_train, y_test)
XGBClassifier_model(X_train_boruta_shap, X_test_boruta_shap, y_train, y_test)

DecisionTreeClassifier_model_(X_train_boruta_shap_with_smoking, X_test_boruta_shap_with_smoking, y_train, y_test)
RandomForestClassifier_model_(X_train_boruta_shap_with_smoking, X_test_boruta_shap_with_smoking, y_train, y_test)
GradientBoostingClassifier_model_(X_train_boruta_shap_with_smoking, X_test_boruta_shap_with_smoking, y_train, y_test)
LGBMClassifier_model_(X_train_boruta_shap_with_smoking, X_test_boruta_shap_with_smoking, y_train, y_test)
XGBClassifier_model_(X_train_boruta_shap_with_smoking, X_test_boruta_shap_with_smoking, y_train, y_test)

0.9569922974032563
0.8916219740788252
0.9662342342342342
0.9553087729548821
0.953970023535241
0.9694562223671812
0.9162787439659666
0.9604232804232805
0.94987184730743
0.9535175996785618


In [175]:
all_data

[['Classifier', 'Without_Smoking', 'With_Smoking'],
 ['DecisionTreeClassifier', 0.9569922974032563, 0.9694562223671812],
 ['RandomForestClassifier', 0.8916219740788252, 0.9162787439659666],
 ['GradientBoostingClassifier', 0.9662342342342342, 0.9604232804232805],
 ['LGBMClassifier', 0.9553087729548821, 0.94987184730743],
 ['XGBClassifier_model', 0.953970023535241, 0.9535175996785618]]

In [176]:
table1 = tabulate(all_data)
table2 = tabulate(all_data,headers='firstrow')

In [177]:
print(tabulate(all_data,headers='firstrow',tablefmt='grid'))

+----------------------------+-------------------+----------------+
| Classifier                 |   Without_Smoking |   With_Smoking |
+============================+===================+================+
| DecisionTreeClassifier     |          0.956992 |       0.969456 |
+----------------------------+-------------------+----------------+
| RandomForestClassifier     |          0.891622 |       0.916279 |
+----------------------------+-------------------+----------------+
| GradientBoostingClassifier |          0.966234 |       0.960423 |
+----------------------------+-------------------+----------------+
| LGBMClassifier             |          0.955309 |       0.949872 |
+----------------------------+-------------------+----------------+
| XGBClassifier_model        |          0.95397  |       0.953518 |
+----------------------------+-------------------+----------------+
